#Importing Dependencies

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import nltk
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import reuters
from nltk.corpus import stopwords
from nltk import PorterStemmer, WordNetLemmatizer
import re
import string

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np


import torch
from torch import optim
from torch import nn
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import hamming_loss, f1_score
from sklearn.preprocessing import MultiLabelBinarizer

from zipfile import ZipFile
import pickle
import time
import copy

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#Function

In [6]:
#Build adjacency matrix based on Co-Occurencies label
def buildAdjacencyCOOC(data_label):
  adj = data_label.T.dot(data_label).astype('float')
  for i in range(len(adj)):
    adj[i] = adj[i] / adj[i,i]

  return torch.from_numpy(adj.astype('float32'))

stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

#Text cleaning function
def preprocessingText(text, stop=stop):
  text = text.lower() #text to lowercase
  text = re.sub(r'&lt;', '', text) #remove '&lt;' tag
  text = re.sub(r'<.*?>', '', text) #remove html
  text = re.sub(r'[0-9]+', '', text) #remove number
  text = " ".join([word for word in text.split() if word not in stop]) #remove stopwords
  text = re.sub(r'[^\w\s]', '', text) #remove punctiation
  text = re.sub(r'[^\x00-\x7f]', '', text) #remove non ASCII strings
  for c in ['\r', '\n', '\t'] :
    text = re.sub(c, ' ', text) #replace newline and tab with tabs
  text = re.sub('\s+', ' ', text) #replace multiple spaces with one space
  text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
  return text

#Load Word Representation Vector
def loadWRVModel(File):
    print("Loading Word Representation Vector Model")
    f = open(File,'r')
    WRVModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        try:
          wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        except:
          print(splitLines[1:])
          print(len(splitLines[1:]))
          break
        WRVModel[word] = wordEmbedding
    print(len(WRVModel)," words loaded!")
    return WRVModel


def check_accuracy(model, label_embedding, X, y):

  model.eval()

  with torch.no_grad():
    out = model(X, label_embedding)
    y_pred = torch.sigmoid(out.detach()).round().cpu()
    f1score = f1_score(y, y_pred, average='micro')
    hammingloss = hamming_loss(y, y_pred)

  return hammingloss, f1score

def train(model,
          X_train,
          X_test,
          label_embedding,
          y_train,
          y_test,
          total_epoch=250,
          batch_size=250,
          learning_rate=0.001,
          save_path='./model.pt',
          state=None):

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  train_data = DataLoader(dataset(X_train, y_train), batch_size=batch_size)
  X_test = X_test.to(device)

  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  criterion = nn.BCEWithLogitsLoss()

  label_embedding= label_embedding.to(device)

  if state:
    state = torch.load(state)
    model = model.load_state_dict(state['last_model'])
    optimizer = optimizer.load_state_dict(state['optimizer'])

  else:
    model = model.to(device)
    state = dict()
    state['microf1'] = []
    state['hammingloss'] = []
    state['val_hammingloss'] = []
    state['val_microf1'] = []
    state['epoch_time'] = []

  epoch = 1

  best_train = 0
  best_val = 0

  while epoch <= total_epoch:
    running_loss = 0
    y_pred = []
    epoch_time = 0
    model.train()
    for index, (X, y) in enumerate(train_data):

      t = time.time()

      #forward
      out = model(X.to(device), label_embedding)
      loss = criterion(out, y.to(device))

      #backward
      optimizer.zero_grad()
      loss.backward()
      clip_grad_norm_(model.parameters(), max_norm=10)

      #update
      optimizer.step()

      epoch_time += time.time() - t
      y_pred.append(torch.sigmoid(out.detach()).round().cpu())
      running_loss += loss.item()

    y_pred = torch.vstack(y_pred)
    f1score = f1_score(y_train, y_pred, average='micro')
    hammingloss = hamming_loss(y_train, y_pred)
    val_hamming, val_f1score = check_accuracy(model, label_embedding, X_test, y_test)

    state['microf1'].append(f1score)
    state['hammingloss'].append(hammingloss)
    state['val_microf1'].append(val_f1score)
    state['epoch_time'].append(epoch_time)
    state['val_hammingloss'].append(val_hamming)

    state['optimizer'] = optimizer.state_dict()
    state['last_model'] = model.state_dict()


    if(best_train < f1score):
      state['model_best_train'] = copy.deepcopy(model.state_dict())
      best_train = f1score
      state['best_train'] = best_train

    if(best_val < val_f1score):
      state['model_best_val'] = copy.deepcopy(model.state_dict())
      best_val = val_f1score
      state['best_val'] = best_val

    torch.save(state, save_path)
    print('epoch:{} loss:{:.5f} hamming_loss:{:.5f} micro_f1score:{:.5f} val_hamming_loss:{:.5f} val_micro_f1score:{:.5f}'.
          format(epoch, running_loss, hammingloss, f1score, val_hamming, val_f1score))
    epoch+=1

#Load Data

## Load Raw Dataset (Reuters-21578)

In [7]:
data_train = pd.read_pickle('./train.pickle')
data_test = pd.read_pickle('./test.pickle')

text_train = data_train.text.values
text_test = data_test.text.values

y_train = torch.from_numpy(np.vstack(data_train.onehot_label.values)).float()
y_test = torch.from_numpy(np.vstack(data_test.onehot_label.values)).float()

print('Train label shape {}'.format(y_train.shape))
print('Test label shape {}'.format(y_test.shape))

Train label shape torch.Size([7769, 90])
Test label shape torch.Size([3019, 90])


#Load MultilabelBinarizer

In [8]:
with open('./multilabelbinarizer.pickle', 'rb') as file:
  mlb = pickle.load(file)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 0.22.2.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


#Build Word Representation Vector Dictionary

In [9]:
WRVModel = loadWRVModel('./glove.6B.300d.txt')

Loading Word Representation Vector Model
32366  words loaded!


#Text Preprocessing

## Text Cleaning

In [10]:
preprocessed_text_train = [preprocessingText(text) for text in text_train]
preprocessed_text_test = [preprocessingText(text) for text in text_test]

print('BEFORE CLEANING: {}'.format(text_train[0]))
print('AFTER CLEANING: {}'.format(preprocessed_text_train[0]))

BEFORE CLEANING: BAHIA COCOA REVIEW
  Showers continued throughout the week in
  the Bahia cocoa zone, alleviating the drought since early
  January and improving prospects for the coming temporao,
  although normal humidity levels have not been restored,
  Comissaria Smith said in its weekly review.
      The dry period means the temporao will be late this year.
      Arrivals for the week ended February 22 were 155,221 bags
  of 60 kilos making a cumulative total for the season of 5.93
  mln against 5.81 at the same stage last year. Again it seems
  that cocoa delivered earlier on consignment was included in the
  arrivals figures.
      Comissaria Smith said there is still some doubt as to how
  much old crop cocoa is still available as harvesting has
  practically come to an end. With total Bahia crop estimates
  around 6.4 mln bags and sales standing at almost 6.2 mln there
  are a few hundred thousand bags still in the hands of farmers,
  middlemen, exporters and processors.
    

##Text to Sequences

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_text_train)

sequences_text_train = tokenizer.texts_to_sequences(preprocessed_text_train)
sequences_text_test = tokenizer.texts_to_sequences(preprocessed_text_test)

X_train = torch.from_numpy(pad_sequences(sequences_text_train, maxlen=128))
X_test = torch.from_numpy(pad_sequences(sequences_text_test, maxlen=128))

## Build Embedding Matrix

In [12]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
embedding_matrix = torch.zeros(VOCAB_SIZE, 300)

unk = 0
for i in range(1, VOCAB_SIZE):
  word = tokenizer.index_word[i]
  if word in WRVModel.keys():
    tensor = torch.from_numpy(WRVModel[word]).float()
    if tensor.size(0)<300:
      # Calculate the number of zeros to add
      num_zeros = 300 - tensor.size(0)

      # Pad the tensor with zeros to reach a size of 300
      embedding_matrix[i] = torch.cat((tensor, torch.zeros(num_zeros)), dim=0)
    elif tensor.size(0)>300:
      embedding_matrix[i] = tensor[:300]
    else:
      embedding_matrix[i]= tensor
  else:
    unk +=1
print('VOCAB_SIZE : {}'.format(VOCAB_SIZE))
print('TOTAL OF UNKNOWN WORD : {}'.format(unk))

VOCAB_SIZE : 25306
TOTAL OF UNKNOWN WORD : 13311


#Preparing Graph Attention Networks Input

## Label Embedding

In [13]:
label_embedding = torch.zeros(90,300)

for index, label in enumerate(mlb.classes_):
  words = label.split('-')
  num_of_words = len(words)

  for sublabel in words:
    if sublabel in WRVModel.keys():
      label_embedding[index] +=  torch.from_numpy(WRVModel[sublabel])
  label_embedding[index] = label_embedding[index]/num_of_words

print(label_embedding)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3568, -0.1348,  0.0790,  ..., -0.0384,  0.2948,  0.1996],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.5990, -0.3234, -0.2749,  ...,  0.6343,  0.5300,  0.0299],
        [-0.4541, -0.1300, -0.5178,  ..., -1.1637, -0.2056, -0.3177]])


##Adjacency Matrix

In [14]:
adjacency = buildAdjacencyCOOC(y_train.numpy())
print(adjacency)

tensor([[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0012],
        [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0286],
        [0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        [0.0952, 0.0476, 0.0000,  ..., 0.0000, 0.0000, 1.0000]])


#Preparing DataLoader and Model

## Dataset Class

In [15]:
class dataset(Dataset):
  def __init__(self, x, y):
    self.x  = x
    self.y = y

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

##Initialize Model

In [16]:
from google.colab import files
src = list(files.upload().values())[0]
open('models_graphsage.py','wb').write(src)
import models_graphsage

Saving models_graphsage.py to models_graphsage (5).py


In [17]:
from models_graphsage import MAGNET

In [21]:
model = MAGNET(300, 250, adjacency, embedding_matrix)

#Model Training

##Configure Save PATH

In [19]:
save_path = './train_result.pt'

## Train Model

In [20]:
train(model, X_train, X_test, label_embedding, y_train, y_test, save_path=save_path)

epoch:1 loss:3.39327 hamming_loss:0.02995 micro_f1score:0.04033 val_hamming_loss:0.01219 val_micro_f1score:0.22792
epoch:2 loss:1.58545 hamming_loss:0.01204 micro_f1score:0.28118 val_hamming_loss:0.01069 val_micro_f1score:0.40214
epoch:3 loss:1.38908 hamming_loss:0.01079 micro_f1score:0.38349 val_hamming_loss:0.01040 val_micro_f1score:0.43024
epoch:4 loss:1.29980 hamming_loss:0.01027 micro_f1score:0.40925 val_hamming_loss:0.01005 val_micro_f1score:0.43054
epoch:5 loss:1.29742 hamming_loss:0.01064 micro_f1score:0.37639 val_hamming_loss:0.01024 val_micro_f1score:0.41134
epoch:6 loss:1.20651 hamming_loss:0.01046 micro_f1score:0.40141 val_hamming_loss:0.01010 val_micro_f1score:0.43268
epoch:7 loss:1.05118 hamming_loss:0.00936 micro_f1score:0.51737 val_hamming_loss:0.00771 val_micro_f1score:0.62248
epoch:8 loss:1.10468 hamming_loss:0.01002 micro_f1score:0.47247 val_hamming_loss:0.00880 val_micro_f1score:0.53726
epoch:9 loss:1.02059 hamming_loss:0.00913 micro_f1score:0.53971 val_hamming_loss

In [ ]:
print(MAGNET.predict("dffgfgbfdbdbfn dfbngvgfhcgn"))